In [1]:
import csv
import pandas as pd
import configparser
from sqlalchemy import create_engine, text

In [2]:
from mysql.connector import MySQLConnection, Error

In [3]:
#Making connection to SQL server
def make_connection():
    try:
        conn = MySQLConnection(host='IES-ADS-ClassDB.sjsu.edu',
                               database='querycrew_db',
                               user='querycrew_user',
                               password='Pomegranate_746')
        
        if conn.is_connected():
            print('Connected to the MySQL database!')
            
            return conn
                
    except Error as e:
        print('Connection failed.')
        print(e)
        
        return None

In [4]:
conn = make_connection()
cursor = conn.cursor()

Connected to the MySQL database!


In [5]:
cursor.execute('DROP TABLE IF EXISTS location')
cursor.execute('DROP TABLE IF EXISTS vehicle')

In [6]:
#This is for the one-to-one tables. The primary key is VIN(1-10) 
#Creating a vehicle table

sql = ( """
        CREATE TABLE vehicle
        (
            vin varchar(32),
            make varchar(32),
            model varchar(32),
            model_year int,
            PRIMARY KEY(vin)
        )
        """
      )
cursor.execute(sql)

In [7]:
#FIGURE OUT A PRIMARY KEY FOR THIS TABLE
sql = ( """
        CREATE TABLE location
        (
            dol_id int,
            vin varchar(20),
            county varchar(32),
            city varchar(32),
            state varchar(32),
            zip_code int,
            PRIMARY KEY(dol_id),
            FOREIGN KEY (vin) REFERENCES vehicle(vin)
        )
        """
      )

cursor.execute(sql);

In [8]:
#For both of these two tables created, the primary key is the vin, and the foriegn key is in the location table and it is the vin as well
sql_insert_vehicle = (  
    """
    INSERT INTO vehicle (vin, model_year, make, model)
    VALUES (%s, %s, %s, %s)
    """
)

In [9]:
sql_insert_location = ( 
    """
    INSERT INTO location (dol_id, vin, county, city, state, zip_code)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
)

In [10]:
def transform_vehicle(row):
    #Converting the model_year column and zip-code column to an int
    try:
      row[1] = int(row[1])
    except ValueError:
      row[4] = None
      row[1] = int(row[1])

    return row

In [11]:
def transform_location(row):
    try:
      row[0] = int(row[0])
      row[4] = int(row[4])
    except ValueError:
      row[0] = None
      row[4] = None

    return row

In [12]:
import csv

vehicle_cols = [0, 5, 6, 7]
location_cols = [13, 0, 1, 2, 3, 4]

first = True
seen_vins = set()

with open('Electric_Vehicle_Population_Data.csv', newline='') as csv_file:
    reader = csv.reader(csv_file, delimiter=',', quotechar='"')
    
   #This is to skip the header row
    next(reader, None) 

    data = []
    for i, row in enumerate(reader):
        # Stopping after reading 100 data rows
        if i >= 100:  
            break
        data.append(row)
for row in data:
    if not first:
        vin = row[vehicle_cols[0]]  

        # Skips over the duplicates found in the file
        if vin in seen_vins:
            continue  
        seen_vins.add(vin)  

        # Here I am checking if the VIN already exists within the database for vehicle
        cursor.execute("SELECT COUNT(*) FROM vehicle WHERE vin = %s", (vin,))
        if cursor.fetchone()[0] > 0:
            continue 
        #This is where we are identifying the specific columns 
        vehicle_row = [row[i] for i in vehicle_cols]
        location_row = [row[i] for i in location_cols]
          
        #Inserting into repsective tables
        cursor.execute(sql_insert_vehicle, vehicle_row)
        cursor.execute(sql_insert_location, location_row)

    first = False  

conn.commit()

In [13]:
sql = (  
    """
    SELECT * FROM vehicle LIMIT 25
    """
)
cursor.execute(sql)
results = cursor.fetchall()



for row in results:
    print(row)

conn.commit()

('1C4JJXP64M', 'JEEP', 'WRANGLER', 2021)
('1C4JJXP65P', 'JEEP', 'WRANGLER', 2023)
('1C4JJXR61P', 'JEEP', 'WRANGLER', 2023)
('1C4RJXN60R', 'JEEP', 'WRANGLER', 2024)
('1FT6W1EV6P', 'FORD', 'F-150', 2023)
('1G1FW6S03H', 'CHEVROLET', 'BOLT EV', 2017)
('1G1FX6S07J', 'CHEVROLET', 'BOLT EV', 2018)
('1G1FX6S08K', 'CHEVROLET', 'BOLT EV', 2019)
('1G1FY6S01N', 'CHEVROLET', 'BOLT EUV', 2022)
('1G1FZ6S05K', 'CHEVROLET', 'BOLT EV', 2019)
('1G1FZ6S06L', 'CHEVROLET', 'BOLT EV', 2020)
('1N4AZ0CP4E', 'NISSAN', 'LEAF', 2014)
('1N4AZ0CP5D', 'NISSAN', 'LEAF', 2013)
('1N4AZ1BP3L', 'NISSAN', 'LEAF', 2020)
('1N4AZ1CP7K', 'NISSAN', 'LEAF', 2019)
('1N4AZ1CPXL', 'NISSAN', 'LEAF', 2020)
('1N4BZ0CP8H', 'NISSAN', 'LEAF', 2017)
('1N4BZ1CP3K', 'NISSAN', 'LEAF', 2019)
('1N4BZ1CP4K', 'NISSAN', 'LEAF', 2019)
('1N4CZ1CV0P', 'NISSAN', 'LEAF', 2023)
('2C4RC1L7XJ', 'CHRYSLER', 'PACIFICA', 2018)
('3C3CFFGE5F', 'FIAT', '500', 2015)
('3MW39FF06R', 'BMW', '330E', 2024)
('5UX43EU06R', 'BMW', 'X5', 2024)
('5UXTA6C04M', 'BMW', 'X5

In [14]:
sql = (  
    """
    SELECT * FROM location LIMIT 25
    """
)
cursor.execute(sql)
results = cursor.fetchall()

if results:
    for row in results:
        print(row)
conn.commit()

(3273763, '5YJSA1E21J', 'King', 'Sammamish', 'WA', 98074)
(3572089, 'WBY1Z4C58F', 'Kitsap', 'Seabeck', 'WA', 98380)
(4735426, '1G1FX6S08K', 'Snohomish', 'Lake Stevens', 'WA', 98258)
(5470846, '5YJXCDE25H', 'King', 'Sammamish', 'WA', 98075)
(6097171, '5YJXCBE29G', 'King', 'Sammamish', 'WA', 98074)
(8874211, 'KMHC65LD9L', 'Kitsap', 'Bremerton', 'WA', 98312)
(105033995, '1N4AZ1CPXL', 'Kitsap', 'Suquamish', 'WA', 98392)
(110038288, '5YJ3E1EA5K', 'Snohomish', 'Lynnwood', 'WA', 98087)
(111162012, '5YJSA1DP8D', 'Snohomish', 'Stanwood', 'WA', 98292)
(111690687, 'JN1AZ0CP7C', 'Snohomish', 'Snohomish', 'WA', 98290)
(112744838, '5YJ3E1EB2L', 'Kitsap', 'Port Orchard', 'WA', 98367)
(118167990, '5YJSA1S23F', 'Snohomish', 'Edmonds', 'WA', 98020)
(122291851, '1N4AZ1BP3L', 'Kitsap', 'Poulsbo', 'WA', 98370)
(124633715, '5YJYGDEE8L', 'Thurston', 'Tumwater', 'WA', 98501)
(126152974, '5YJ3E1EB0L', 'Snohomish', 'Bothell', 'WA', 98012)
(127281031, '5YJ3E1EA4L', 'Snohomish', 'Lake Stevens', 'WA', 98258)
(1350

In [15]:
df = pd.read_csv('Electric_Vehicle_Population_Data.csv')

# Extracting 50 rows of "Make" and "Model Year" from CSV (using .loc for label indexing the CSV file)
#We only want the UNIQUE values of Make to present the One-to-Many Relationship with the second table.
make_and_model_year_data = df.loc[:49, ["Make", "Model Year"]].groupby("Make").first().reset_index()

# Extracting 50 rows of "Make", "Model", and "Vehicle Type" from CSV file.
make_model_type_data = df.loc[:49, ["Make", "Model", "Electric Vehicle Type"]]

print("Make and Model Year Data")
print(make_and_model_year_data.head(25))
 
print("\n Make Model Type Data")
print(make_model_type_data.head(25))

Make and Model Year Data
         Make  Model Year
0         BMW        2015
1   CHEVROLET        2019
2        FIAT        2015
3     HYUNDAI        2022
4        JEEP        2023
5         KIA        2024
6       LEXUS        2023
7       MAZDA        2024
8      NISSAN        2017
9    POLESTAR        2021
10     RIVIAN        2022
11      TESLA        2019
12     TOYOTA        2018
13      VOLVO        2022

 Make Model Type Data
         Make     Model                   Electric Vehicle Type
0        JEEP  WRANGLER  Plug-in Hybrid Electric Vehicle (PHEV)
1   CHEVROLET   BOLT EV          Battery Electric Vehicle (BEV)
2         BMW        I3          Battery Electric Vehicle (BEV)
3       TESLA   MODEL 3          Battery Electric Vehicle (BEV)
4       TESLA   MODEL S          Battery Electric Vehicle (BEV)
5        JEEP  WRANGLER  Plug-in Hybrid Electric Vehicle (PHEV)
6         BMW      530E  Plug-in Hybrid Electric Vehicle (PHEV)
7       TESLA   MODEL X          Battery Electric 

In [ ]:
#Drop table if it already exists
cursor.execute('DROP TABLE IF EXISTS make_model_year')
cursor.execute('DROP TABLE IF EXISTS make_model_type')

#Creating the tables to hold make, year, model, and type of vehicles
make_model_year = ( """
        CREATE TABLE make_model_year
        (
            make       varchar(32),
            year      int,
            PRIMARY KEY (make)
        )
        """
      )
make_model_type = ( """
        CREATE TABLE make_model_type
        (
            make       varchar(32),
            model   varchar(32),
            type        varchar(64)
        )
        """
      )

cursor.execute(make_model_year);
cursor.execute(make_model_type);

print("Tables created successfully!")

In [ ]:
#Inserting values into the respective tables
sql_insert_make_model_year = (  
    """
    INSERT INTO make_model_year (make, year)
    VALUES (%s, %s)
    """
)

sql_insert_make_model_type = (  
    """
    INSERT INTO make_model_type (make, model, type)
    VALUES (%s, %s, %s)
    """
)

#Looping and iterating through the respective DataFrames to insert values into created Databases.
for row in make_and_model_year_data.itertuples(index=False, name=None):
    cursor.execute(sql_insert_make_model_year, row)
print("Inserted make and model_year data successfully!")
for row in make_model_type_data.itertuples(index=False, name=None):
    cursor.execute(sql_insert_make_model_type, row)
print("Inserted make, model, and type successfully!")


Inserted make and model_year data successfully!
Inserted make, model, and type successfully!


In [ ]:
#Displaying the one-to-many relationship
#where one MAKE can have MULTIPLE MODELS

print("Make-Model-Year Table")
sql_make_model_year = (
    """
    SELECT * FROM make_model_year LIMIT 25;
    """
)
cursor.execute(sql_make_model_year)
result_data = cursor.fetchall()
for r in result_data:
    print(r)

print("\nMake-Model-Type Table")
sql_make_model_type = (
    """
    SELECT * FROM make_model_type LIMIT 25;
    """
)
cursor.execute(sql_make_model_type)
result_data_2 = cursor.fetchall()
for r2 in result_data_2:
    print(r2)


Make-Model-Year Table
('BMW', 2015)
('CHEVROLET', 2019)
('FIAT', 2015)
('HYUNDAI', 2022)
('JEEP', 2023)
('KIA', 2024)
('LEXUS', 2023)
('MAZDA', 2024)
('NISSAN', 2017)
('POLESTAR', 2021)
('RIVIAN', 2022)
('TESLA', 2019)
('TOYOTA', 2018)
('VOLVO', 2022)

Make-Model-Type Table
('JEEP', 'WRANGLER', 'Plug-in Hybrid Electric Vehicle (PHEV)')
('CHEVROLET', 'BOLT EV', 'Battery Electric Vehicle (BEV)')
('BMW', 'I3', 'Battery Electric Vehicle (BEV)')
('TESLA', 'MODEL 3', 'Battery Electric Vehicle (BEV)')
('TESLA', 'MODEL S', 'Battery Electric Vehicle (BEV)')
('JEEP', 'WRANGLER', 'Plug-in Hybrid Electric Vehicle (PHEV)')
('BMW', '530E', 'Plug-in Hybrid Electric Vehicle (PHEV)')
('TESLA', 'MODEL X', 'Battery Electric Vehicle (BEV)')
('CHEVROLET', 'BOLT EV', 'Battery Electric Vehicle (BEV)')
('TESLA', 'MODEL 3', 'Battery Electric Vehicle (BEV)')
('NISSAN', 'LEAF', 'Battery Electric Vehicle (BEV)')
('LEXUS', 'RZ', 'Battery Electric Vehicle (BEV)')
('KIA', 'EV6', 'Battery Electric Vehicle (BEV)')
('N

In [ ]:
config = configparser.ConfigParser()
config.read("querycrew.ini")

# Extract MySQL details
db_host = config["mysql"]["host"]
db_user = config["mysql"]["user"]
db_password = config["mysql"]["password"]
db_name = config["mysql"]["database"]

# Connect to MySQL
engine = create_engine(f"mysql+mysqlconnector://{db_user}:{db_password}@{db_host}/{db_name}")


In [ ]:
# Select and clean Makes data (limit to 100 rows)
make_data = df[["Make"]].drop_duplicates().head(100)
make_data.columns = ["make_name"]

# Select and clean Counties data (limit to 100 rows)
county_data = df[["County"]].drop_duplicates().head(100)
county_data.columns = ["county_name"]

# Extract Make-County Relationships (limit to 100 rows)
make_county_data = df[["Make", "County"]].drop_duplicates().head(100)
make_county_data.columns = ["make_name", "county_name"]

# Display data
print("Makes Data (First 25 Rows):")
print(make_data.head(25))

print("\nCounties Data (First 25 Rows):")
print(county_data.head(25))

print("\nMake-County Relationships (First 25 Rows):")
print(make_county_data.head(25))

Makes Data (First 25 Rows):
         make_name
0             JEEP
1        CHEVROLET
2              BMW
3            TESLA
10          NISSAN
11           LEXUS
12             KIA
17        POLESTAR
19         HYUNDAI
25          TOYOTA
27          RIVIAN
28           VOLVO
30            FIAT
40           MAZDA
55            AUDI
79        CHRYSLER
92            FORD
136  MERCEDES-BENZ
161          HONDA
165     MITSUBISHI
240     VOLKSWAGEN
242         SUBARU
269        PORSCHE
333         JAGUAR
500           MINI

Counties Data (First 25 Rows):
        county_name
0            Kitsap
1         Snohomish
2              King
4            Yakima
6          Thurston
17           Island
95          Stevens
107           Grant
130        Kittitas
148         Spokane
150         Whitman
151          Skagit
156         Douglas
169     Walla Walla
175          Chelan
225      Williamson
253           Clark
260         Cowlitz
264       Jefferson
1061      Klickitat
2262        Clallam
37687 

In [ ]:
with engine.connect() as conn:
    # Drop existing tables
    conn.execute(text("DROP TABLE IF EXISTS make_counties;"))
    conn.execute(text("DROP TABLE IF EXISTS makes;"))
    conn.execute(text("DROP TABLE IF EXISTS counties;"))

    # Create Makes Table
    
    conn.execute(text("""
        CREATE TABLE makes (
            make_id INT AUTO_INCREMENT PRIMARY KEY,  -- makes_id is the Primary Key for makes table
            make_name VARCHAR(50) UNIQUE NOT NULL    -- Make names must be unique
        )
    """))

    # Create Counties Table
    conn.execute(text("""
        CREATE TABLE counties (
            county_id INT AUTO_INCREMENT PRIMARY KEY,  -- county_id is the Primary Key for counties table
            county_name VARCHAR(100) UNIQUE NOT NULL   -- Each county has a unique name
        )
    """))

    # Create Make-County Linking Table
    conn.execute(text("""
        CREATE TABLE make_counties (
            make_id INT,    -- Foreign Key referencing the makes table
            county_id INT,  -- Foreign Key referencing the counties table
            PRIMARY KEY (make_id, county_id),  -- Composite Primary Key that ensures unique pairs
            FOREIGN KEY (make_id) REFERENCES makes(make_id) ON DELETE CASCADE,  -- Deletes linked records if make is removed
            FOREIGN KEY (county_id) REFERENCES counties(county_id) ON DELETE CASCADE  -- Ensures valid county refrences
        )
    """))

    print("Tables created successfully!")


Tables created successfully!


In [ ]:
# Insert Makes Data
with engine.connect() as conn:
    for _, row in make_data.iterrows():
        conn.execute(text("""
            INSERT IGNORE INTO makes (make_name) 
            VALUES (:make_name)
        """), row.to_dict())

    print("Makes inserted successfully!")

# Insert Counties Data
with engine.connect() as conn:
    for _, row in county_data.iterrows():
        conn.execute(text("""
            INSERT IGNORE INTO counties (county_name) 
            VALUES (:county_name)
        """), row.to_dict())

    print("Counties inserted successfully!")

# Insert Make-County Relationships
with engine.connect() as conn:
    for _, row in make_county_data.iterrows():
        conn.execute(text("""
            INSERT IGNORE INTO make_counties (make_id, county_id)
            SELECT m.make_id, c.county_id
            FROM makes m, counties c
            WHERE m.make_name = :make_name AND c.county_name = :county_name
        """), row.to_dict())

    print("Make-County relationships inserted successfully!")


Makes inserted successfully!
Counties inserted successfully!
Make-County relationships inserted successfully!


In [ ]:
# Query and display first 25 rows from each table
with engine.connect() as conn:
    print("\nMakes Table (First 25 Rows):")
    result = conn.execute(text("SELECT * FROM makes LIMIT 25;"))
    for row in result:
        print(row)

    print("\nCounties Table (First 25 Rows):")
    result = conn.execute(text("SELECT * FROM counties LIMIT 25;"))
    for row in result:
        print(row)

    print("\nMake-County Relationships (First 25 Rows):")
    result = conn.execute(text("""
        SELECT m.make_name, c.county_name 
        FROM make_counties mc
        JOIN makes m ON mc.make_id = m.make_id
        JOIN counties c ON mc.county_id = c.county_id
        LIMIT 25;
    """))
    for row in result:
        print(row)



Makes Table (First 25 Rows):

Counties Table (First 25 Rows):

Make-County Relationships (First 25 Rows):


In [ ]:
#Here are the 5 sql queries joining the above tables:

queries = [
    "SELECT v.vin, v.make, v.model, v.model_year, m.year FROM vehicle v JOIN make_model_year m ON v.make = m.make;",
    "SELECT v.vin, v.make, v.model, mmt.type FROM vehicle v JOIN make_model_type mmt ON v.make = mmt.make AND v.model = mmt.model;",
    "SELECT mk.make_name, c.county_name FROM makes mk JOIN make_counties mc ON mk.make_id = mc.make_id JOIN counties c ON mc.county_id = c.county_id;",
    "SELECT v.make, COUNT(v.vin) AS total_vehicles FROM vehicle v GROUP BY v.make;",
    "SELECT v.vin, v.make, v.model, v.model_year, c.county_name FROM vehicle v JOIN makes mk ON v.make = mk.make_name JOIN make_counties mc ON mk.make_id = mc.make_id JOIN counties c ON mc.county_id = c.county_id WHERE c.county_name = 'Los Angeles County';"
]

with engine.connect() as connection:
    for query in queries:
        result = connection.execute(text(query))
        print(result.fetchall())

[]
[]
[]
[('JEEP', 4), ('FORD', 1), ('CHEVROLET', 6), ('NISSAN', 12), ('CHRYSLER', 1), ('FIAT', 1), ('BMW', 9), ('TESLA', 39), ('RIVIAN', 3), ('MAZDA', 2), ('TOYOTA', 1), ('LEXUS', 2), ('HYUNDAI', 2), ('KIA', 7), ('POLESTAR', 1), ('AUDI', 2), ('VOLVO', 2)]
[]
